<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import spacy
import wikipedia
import pandas as pd
from spacy import displacy

import urllib 
import time
import json
import requests 
from bs4 import BeautifulSoup

In [2]:
# read json file of wikipedia page title and summary

with open('../01_make_matching_list/title_summary.json', 'r') as f:
    DICT_PAGE_TITLE = json.load(f)

In [5]:
def find_wiki_title(term):
    title = wikipedia.search(term)
    if title != []:
        return title[0]

    
def find_wiki_summary(term):
    try:
        return wikipedia.summary(term)
    # if it is a ambiguous term, the function will return None as value of summary
    except wikipedia.exceptions.DisambiguationError:
        return None


def text_to_wiki(text, nlp): # gives the html under BeatifulSoup format
    doc = nlp(text)

    html = displacy.render(doc, style="ent", options={"ents": ["TERM"]}, jupyter=False, page=True)
    soup = BeautifulSoup(html)
    marks =  soup.find_all('mark')
    
    for mark in marks :
        try:
            term = mark.get_text(strip=True,separator=', ').split(', ')[0] # get the term annotated
            
            wiki_info = DICT_PAGE_TITLE[term] # get wikipedia pagetitle and summary from json file
            url = f'https://en.wikipedia.org/wiki/{"_".join(wiki_info["redirect"].split())}' 
            summary = wiki_info['summary']
            
        except KeyError:
            wiki_title = find_wiki_title(term)   
            if wiki_title:
                url = f'https://en.wikipedia.org/wiki/{"_".join(wiki_title.split())}' 
                DICT_PAGE_TITLE.update({term:{'title':wiki_title, 'summary': wiki_summary}})
                    
        link = soup.new_tag('a', href=url) # create the html tag for link       
        mark.wrap(link) #add html tag <a> (the one to make links) to around our annotated word
    return soup

In [6]:
nlp = spacy.load(r"./output/G06F_2019/model-best/") #load the best model

with open('../G06T0007254000.txt') as f:
    patents = f.read()


doc = nlp(patents) # input sample text

OSError: [E050] Can't find model './output/G06F_2019/model-best/'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
open('./demo.html', 'w', encoding="utf-8").write(str(text_to_wiki(patents,nlp))) #create file with html data

In [ ]:
# save new json 
with open(DICT_PAGE_TITLE, "w") as outfile: 
    json.dump(wiki_title_dict, outfile, indent = 4)